# Compute and Request Features with VUMT

## 1. Compute features 

In [ ]:
from zizou.dsar import DSAR
from zizou.rsam import RSAM, EnergyExplainedByRSAM
from zizou.ssam import SSAM
from zizou.spectral_features import SpectralFeatures
from zizou.data import DataSource, MockSDSWaveforms
from obspy import UTCDateTime

import numpy as np
import os 
from tonik import Storage

In this notebook, we will still be using our *mock* **DataSource** to generate data and compute **RSAM**. Once computed these features will be saved in a *hdf5* format to be queried later.

In [ ]:

# Instantiate DataSource 
mc = MockSDSWaveforms(sds_dir="/tmp/sds/seismic")
ds = DataSource(clients=[mc], chunk_size=86400)
sg = Storage("Whakaari", rootdir="/tmp/vumt/features")
site = "WIZ"
sensor = "10"
channel = "HHZ"
store = sg.get_substore(site=site, sensor=sensor, channel=channel)

# Instantiate Features 
rsam = RSAM(filtertype='bandpass', filterfreq=(1, 4.9))
rsam_energy_prop = EnergyExplainedByRSAM(filterfreq_wb=(0.5, 4.99))
ssam = SSAM(
    frequencies=np.linspace(0, 4.99, 10),
    timestamp="start", 
    resample_int=(None, "10min")
)
dsar = DSAR(lowerfreqband=(0.1, 2.5), higherfreqband=(2.5, 4.99))
spec = SpectralFeatures()

feature_dir = "/tmp/vumt/features/Whakaari/WIZ/HHZ"
if not os.path.isdir(feature_dir):
    os.makedirs(feature_dir)
    
for tr in ds.get_waveforms(net="NZ", site=site, loc=sensor, comp=channel, start=UTCDateTime("2024-05-01"), end=UTCDateTime("2024-06-01")):
    print(tr)
    for feat in (rsam, rsam_energy_prop, ssam, dsar, spec):
        store.save(feat.compute(trace=tr))


## 2. Request feature once computed

In [ ]:
import datetime

In [ ]:
starttime = datetime.datetime(2024, 5, 10)
endtime = datetime.datetime(2024, 6, 1)

sg.starttime = starttime
sg.endtime = endtime
store = sg.get_substore(site=site, sensor=sensor, channel=channel)
rsam = store("rsam")
ssam = store("ssam")
print(rsam)

## Plot requested data

In [ ]:
import matplotlib.pyplot as plt 
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 5), sharex=True)
ax1.plot(rsam.coords["datetime"], rsam.data)
ax1.set_ylabel("RSAM")
ax2.imshow(ssam.data, extent=[rsam.coords["datetime"][0].values, rsam.coords["datetime"][-1].values, 0, 5])
ax2.set_aspect("auto")
ax2.set_ylabel("SSAM")